In [ ]:
import os
import yfinance as yf
import pandas as pd
from fastai.tabular.all import *

# Define the stock symbol
symbol = 'AAPL'

# Fetch data
data = yf.download(symbol, start="2010-01-01", end="2023-01-11")

# Feature Engineering
data['MA50'] = data['Close'].rolling(window=50).mean()
data['MA200'] = data['Close'].rolling(window=200).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA200'] = data['Close'].ewm(span=200, adjust=False).mean()
data.dropna(inplace=True)

# Model Training
dep_var = 'Close'
features = ['Open', 'High', 'Low', 'Volume', 'MA50', 'MA200', 'EMA50', 'EMA200']

# Split data for training and validation
train_idx = list(range(0, len(data) - 9))
valid_idx = list(range(len(data) - 9, len(data)))

to = TabularPandas(data, procs=[Normalize],
                   cont_names=features,
                   y_names=dep_var,
                   splits=(train_idx, valid_idx))

dls = to.dataloaders(bs=64)

# Define model configuration with reduced dropout
config = tabular_config(ps=[0.2, 0.2])

# Define the learner with a simpler architecture
learn = tabular_learner(dls, layers=[200, 100], config=config, metrics=rmse, y_range=(min(data[dep_var]), max(data[dep_var])))

# Use a slightly higher learning rate for training
lr_selected = 5e-3
learn.fit_one_cycle(10, lr_selected)

# Predictions for January 2-10, 2023
test_data = data.iloc[-9:]
dl = dls.test_dl(test_data)
predictions, actuals = learn.get_preds(dl=dl)

# Create a DataFrame to compare actual vs predicted prices
compare_df = pd.DataFrame({'Actual': actuals.numpy().squeeze(), 'Predicted': predictions.numpy().squeeze()})
print(compare_df)

# Enhanced plotting style with a title and improved aesthetics
plt.figure(figsize=(15, 7))

# Plotting the data
plt.plot(compare_df['Actual'], label='Actual', marker='o', linestyle='-', color='blue')
plt.plot(compare_df['Predicted'], label='Predicted', marker='x', linestyle='--', color='darkorange')

# Setting labels, title, and legend
plt.xlabel('Date (January 2-10, 2023)', fontsize=14)
plt.ylabel('Close Price', fontsize=14)
plt.title('Comparison of Actual vs Predicted Close Prices', fontsize=16, fontweight='bold')
plt.legend(loc='upper left', fontsize=12)

# Enhancing the grid and layout
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()

# Display the plot
plt.show()

